In [35]:
# Ensure we are in the correct directory #
#----------------------------------------#

%cd ~/Documents/contabo/equity2/dashboard/playground/buy_the_dip
%pwd

/Users/ludwigjonsson/Documents/contabo/equity2/dashboard/playground/buy_the_dip


using bookmarks requires you to install the `pickleshare` library.
using dhist requires you to install the `pickleshare` library.


'/Users/ludwigjonsson/Documents/contabo/equity2/dashboard/playground/buy_the_dip'

In [36]:
# Set up current notebook #
#-------------------------#

try: IS_MASTER
except: IS_MASTER = False

if not IS_MASTER:
    DATA_DIR = './data'
    EXP_DIR = './exp'
    PROCESSED_FILE = f'{DATA_DIR}/processed.pkl'
    MODEL_FILE = f'{EXP_DIR}/model.pkl'
    PREDICTION_FILE = f'{EXP_DIR}/ypred.pkl'
    OTHER_MODEL_PARAM = ...


In [37]:
# Import Dependencies #
#---------------------#

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import shap

In [38]:
# Open the CSV file #
#-------------------#

df = pd.read_pickle(PROCESSED_FILE)
df

metric,CloseAdj,Volume,CloseAdj_VOLATIL14,HighLow_Spread,OpenClose_Spread,CloseAdj_VWAP14,CloseAdj_VWAP14_Spread,CloseAdj_ZSCORE22,Volume_ZSCORE22,HighLow_Spread_ZSCORE22,OpenClose_Spread_ZSCORE22,CloseAdj_VWAP14_ZSCORE22,CloseAdj_VWAP14_Spread_ZSCORE22,TradeDirection
Date,,,,,,,,,,,,,,
2014-02-17,22370.000000,10990.0,0.422499,451.380859,337.0,21158.410436,1211.589564,0.790529,-0.328911,0.197243,1.392717,-1.032132,1.742416,0
2014-02-18,22849.000000,4949.0,0.495300,568.000000,529.0,21215.322867,1633.677133,1.403122,-1.008927,0.638729,1.865736,-0.855091,2.042741,0
2014-02-19,22889.000000,18515.0,0.552944,305.910156,39.0,21384.432974,1504.567026,1.482150,0.457418,-0.596890,0.169365,-0.496022,1.708578,0
2014-02-20,22966.000000,9349.0,0.587103,527.699219,418.0,21493.348524,1472.651476,1.589809,-0.649678,0.386809,1.345380,-0.226510,1.530864,0
2014-02-21,23162.000000,9798.0,0.590357,224.847656,-78.0,21711.867483,1450.132517,1.804278,-0.633837,-1.010934,-0.323862,0.303278,1.387709,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14,32795.000000,7042.0,0.222428,566.560547,430.0,32337.008493,457.991507,0.159289,2.174789,0.261543,1.226203,-1.283205,1.118623,1
2023-11-15,33087.500000,4325.0,0.241326,423.398438,-67.5,32423.409572,664.090428,0.650941,0.719807,-0.347473,-0.049624,-1.065807,1.309906,1
2023-11-16,32581.949219,0.0,0.215079,0.000000,0.0,32515.381096,66.568123,0.184333,-1.445229,-1.867283,0.140468,-0.859909,0.772514,1


In [39]:
if not IS_MASTER:
    print(df.columns)

Index(['CloseAdj', 'Volume', 'CloseAdj_VOLATIL14', 'HighLow_Spread',
       'OpenClose_Spread', 'CloseAdj_VWAP14', 'CloseAdj_VWAP14_Spread',
       'CloseAdj_ZSCORE22', 'Volume_ZSCORE22', 'HighLow_Spread_ZSCORE22',
       'OpenClose_Spread_ZSCORE22', 'CloseAdj_VWAP14_ZSCORE22',
       'CloseAdj_VWAP14_Spread_ZSCORE22', 'TradeDirection'],
      dtype='object', name='metric')


In [40]:
# Set up the model and features #
#-------------------------------#

XColumns = [
    'CloseAdj',
    'Volume',
    'CloseAdj_VOLATIL14',
    'HighLow_Spread',
    'OpenClose_Spread',
    'CloseAdj_VWAP14',
    'CloseAdj_VWAP14_Spread',
    'CloseAdj_ZSCORE22',
    'Volume_ZSCORE22',
    'HighLow_Spread_ZSCORE22',
    'OpenClose_Spread_ZSCORE22',
    'CloseAdj_VWAP14_ZSCORE22',
    'CloseAdj_VWAP14_Spread_ZSCORE22'
]

yColumns = ['TradeDirection']

X = df[XColumns].values
y = df[yColumns].values

parameters = {
    'criterion': ['gini'],
    'max_depth': range(2, 22),
    'min_samples_split': range(2, 6),
    'min_samples_leaf': range(1, 6),
    # 'max_leaf_nodes': range(10, 1000, 50),
    'min_impurity_decrease': np.linspace(0, 1, 25),
    'ccp_alpha': np.linspace(0, 1, 25),
    'random_state': [0]
}

model = DecisionTreeClassifier()

In [41]:
# Run the model CV #
#------------------#

clf = GridSearchCV(model, parameters, n_jobs=-1, verbose=1)
clf.fit(X=X, y=y)

Fitting 5 folds for each of 250000 candidates, totalling 1250000 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': array([0.        , 0.04166667, 0.08333333, 0.125     , 0.16666667,
       0.20833333, 0.25      , 0.29166667, 0.33333333, 0.375     ,
       0.41666667, 0.45833333, 0.5       , 0.54166667, 0.58333333,
       0.625     , 0.66666667, 0.70833333, 0.75      , 0.79166667,
       0.83333333, 0.875     , 0.91666667, 0.95833333, 1.        ]),
                         'criterion': ['gini'], 'max_depth': range(2, 22),
                         'min_impurity_decrease': array([0.        , 0.04166667, 0.08333333, 0.125     , 0.16666667,
       0.20833333, 0.25      , 0.29166667, 0.33333333, 0.375     ,
       0.41666667, 0.45833333, 0.5       , 0.54166667, 0.58333333,
       0.625     , 0.66666667, 0.70833333, 0.75      , 0.79166667,
       0.83333333, 0.875     , 0.91666667, 0.95833333, 1.        ]),
                         'min_samples_leaf': range(1, 6),
                         'min_samples_split': range(2, 6),
                         'random_state': [0]},
             verbose=1)

In [45]:
cvResults = pd.DataFrame(clf.cv_results_)
cvResults.sort_values('rank_test_score')

# cvResults.to_pickle('gridcvresult.pkl')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ccp_alpha,param_criterion,param_max_depth,param_min_impurity_decrease,param_min_samples_leaf,param_min_samples_split,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
124999,0.002666,0.000016,0.000248,0.000003,0.5,gini,11,1.0,5,5,0,"{'ccp_alpha': 0.5, 'criterion': 'gini', 'max_d...",0.704453,0.705882,0.703854,0.703854,0.703854,0.704380,0.000786,1
166659,0.003291,0.000846,0.000305,0.000046,0.666667,gini,15,0.291667,5,5,0,"{'ccp_alpha': 0.6666666666666666, 'criterion':...",0.704453,0.705882,0.703854,0.703854,0.703854,0.704380,0.000786,1
166660,0.004522,0.001940,0.000530,0.000294,0.666667,gini,15,0.333333,1,2,0,"{'ccp_alpha': 0.6666666666666666, 'criterion':...",0.704453,0.705882,0.703854,0.703854,0.703854,0.704380,0.000786,1
166661,0.003546,0.001668,0.000268,0.000031,0.666667,gini,15,0.333333,1,3,0,"{'ccp_alpha': 0.6666666666666666, 'criterion':...",0.704453,0.705882,0.703854,0.703854,0.703854,0.704380,0.000786,1
166662,0.003961,0.001314,0.000494,0.000279,0.666667,gini,15,0.333333,1,4,0,"{'ccp_alpha': 0.6666666666666666, 'criterion':...",0.704453,0.705882,0.703854,0.703854,0.703854,0.704380,0.000786,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7510,0.030810,0.004718,0.000602,0.000189,0.0,gini,17,0.0,3,4,0,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.552632,0.509128,0.553753,0.618661,0.738337,0.594502,0.079968,249989
9011,0.020784,0.001703,0.000381,0.000049,0.0,gini,20,0.0,3,5,0,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.552632,0.509128,0.555781,0.616633,0.738337,0.594502,0.079648,249989
7511,0.029467,0.002312,0.001028,0.000590,0.0,gini,17,0.0,3,5,0,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.552632,0.509128,0.553753,0.618661,0.738337,0.594502,0.079968,249989
8510,0.045550,0.011256,0.000953,0.000412,0.0,gini,19,0.0,3,4,0,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.552632,0.509128,0.555781,0.616633,0.738337,0.594502,0.079648,249989
